In [1]:
# installing required SQL magic packages
!pip install sqlalchemy
!pip install ipython-sql
!pip install pymysql

In [2]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [3]:
# connect to sqlite database(if it doesnt exist we crete it)
%reload_ext sql

engine = create_engine('sqlite:///bonga_ecommerce.db')
%config SQL.conn_name = 'engine'

conn = sqlite3.connect('bonga_ecommerce.db')
cursor = conn.cursor()


%sql sqlite:///bonga_ecommerce.db

In [15]:
#create tables
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS products(
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price DECIMAL NOT NULL,
    category TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orderItems(
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')


In [5]:
%%sql
SELECT name FROM sqlite_master WHERE TYPE IN ('table', 'view') AND name NOT LIKE 'sqlite_%' ORDER BY 1

 * sqlite:///bonga_ecommerce.db
Done.


name
customers
orderItems
orders
products


In [14]:
# drop tables
cursor.execute('''
DROP TABLE IF EXISTS products;
''')
cursor.execute('''
DROP TABLE IF EXISTS customers;
''')
cursor.execute('''
DROP TABLE IF EXISTS orders;
''')
cursor.execute('''
DROP TABLE IF EXISTS orderItems;
''')


In [45]:
import pandas as pd
#df = pd.read_csv('./10alytics Data/products.csv')
#print(df[df.duplicated('product_id', keep=False)])  # This will show all entries where `product_id` is duplicated.


In [46]:
#df = df.drop_duplicates(subset='product_id')

In [47]:
#df.to_csv('edited_product.csv', index=False)

In [48]:
df.head()

,product_id,name,price,category
0,1,Product 1,193.52,Fashion
1,2,Product 2,475.85,Home & Kitchen
2,3,Product 3,368.68,Toys
3,4,Product 4,303.34,Electronics
4,5,Product 5,86.45,Toys


In [16]:
# function to load the data into the database table
# Function to load and clean data
def load_data_to_table(csv_path, table_name):
    df = pd.read_csv(csv_path)
    #df.drop_duplicates(subset='product_id', keep='first', inplace=True)  # Remove duplicates
    df.to_sql(table_name, conn, if_exists='append', index=False)

In [17]:
load_data_to_table('products.csv', 'products')
load_data_to_table('customers_bonga.csv', 'customers')
load_data_to_table('orders.csv', 'orders')
load_data_to_table('order_items.csv', 'orderItems')